In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Load the dataset
data = np.load('lab2_dataset.npz')
train_feats = torch.tensor(data['train_feats']) # 44730 examples, 11 frames/example, 40 mfcc bins
test_feats = torch.tensor(data['test_feats']) # 4773 examples, 11 frames/example, 40 mfcc bins
train_labels = torch.tensor(data['train_labels']) # 44730 training data labels 
test_labels = torch.tensor(data['test_labels']) # 4773 testing data labels 
phone_labels = data['phone_labels'] # 48 labels for the 48 classifications

# Set up the dataloaders
train_dataset = torch.utils.data.TensorDataset(train_feats, train_labels)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=True)

test_dataset = torch.utils.data.TensorDataset(test_feats, test_labels)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, shuffle=False)

# Define the model architecture
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        # TODO: Fill in the model's layers here
        self.linear1 = nn.Linear(40, 100) # hidden layer
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(100, 48) #o utput layer

    def forward(self, x):
        # TODO: Fill in the forward pass here
        x = self.relu(self.linear1(x))
        x = self.linear2(x)
        return x

# Instantiate the model, loss function, and optimizer
model = MyModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

def train_network(model, train_loader, criterion, optimizer):
    # TODO: fill in
    numEpochs = 10
    for epoch in range(numEpochs):
        for i, (inputs, labels) in enumerate(train_loader, 0):
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

def test_network(model, test_loader):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print('Test accuracy: %d %%' % (100 * correct / total))

train_network(model, train_loader, criterion, optimizer)
test_network(model, test_loader)


RuntimeError: Expected target size [8, 48], got [8]